In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join

In [2]:
onlyfiles = [f for f in listdir("../measurement/output/20250207_odmr_tracking_snr/") if isfile(join("../measurement/output/20250207_odmr_tracking_snr/", f))]
onlyfiles

['data_dev_pulsed_odmr_tracking_changedetection_1738969052.74927_runlen_10.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738969170.3873248_runlen_60.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738969324.7462828_runlen_120.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738969536.9288688_runlen_180.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738969806.6307354_runlen_240.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738970132.5246866_runlen_300.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738971140.0006485_runlen_600.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1738973427.3779132_runlen_1.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1739303840.0672069_runlen_1.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1739304021.0119052_runlen_100.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1739304349.0267253_runlen_150.pkl',
 'data_dev_pulsed_odmr_tracking_changedetection_1739304760.2964664_runlen_170.pkl',


In [3]:
import re

def extract_runlen_numbers(filenames):
    numbers = []
    pattern = re.compile(r'_runlen_(\d+)\.pkl$')
    
    for filename in filenames:
        match = pattern.search(filename)
        if match:
            numbers.append(int(match.group(1)))
    
    return numbers

In [4]:
print(extract_runlen_numbers(onlyfiles))

[10, 60, 120, 180, 240, 300, 600, 1, 1, 100, 150, 170, 110, 130, 5, 20, 20, 30, 40, 50, 70, 80, 90, 1, 140, 190, 220, 350, 400, 450, 500, 550, 325, 375, 425, 475, 525, 700, 800, 900, 1200]


In [5]:
def read_data(fname):
    try:
        with open(fname, "rb") as file:
            loaded_data = pickle.load(file)
        return loaded_data["buffer_data"], loaded_data["paraset"]
    except (FileNotFoundError, pickle.UnpicklingError) as e:
        print(f"Error loading file {fname}: {e}")
        return None, None
    
# Function to average repeated data
def average_repeated_data(array, initial, end, index, reps):
    subdata=[0,0,0,0]
    for i in range(4):
        arr=array[i]
        arr[:index, :] /= (reps + 1)
        arr[index:] /= reps
        subdata[i]=np.mean(arr[:, initial:end], axis=1)
    return subdata


def add_fft_trace(data, t_atrack):
    x=[0,0,0,0]
    y=[0,0,0,0]
    for i in range(4):
        buffer = data[i]
        freq = np.fft.fftfreq(len(buffer), (t_atrack / 1E9))
        sig_fft = np.abs(np.fft.fft(np.abs(buffer)))
        sig_fft -= np.abs(np.fft.fft(data[0]))
        positive_freqs = freq > 0
        x[i]=freq[positive_freqs]
        y[i]=(sig_fft)[positive_freqs]
    return x,y

def calculate_mean_noise_fft(fft_result,lower,higher,x): 
    loc = np.where((x[3] < lower) | (x[3] > higher))[0]
    magnitude_spectrum = np.abs(fft_result[loc])

    # Calculate the mean of the magnitude spectrum
    mean_magnitude = np.mean(magnitude_spectrum)
    # noise_threshold_multiplier = 0.6
    # # Define a threshold to separate noise from signal
    # noise_threshold = noise_threshold_multiplier * mean_magnitude

    # # Identify noise components (below the threshold)
    # noise_components = magnitude_spectrum[magnitude_spectrum < noise_threshold]

    # # Calculate the mean of the noise components
    # mean_noise = np.mean(noise_components) if noise_components.size > 0 else 0
    
    return mean_magnitude

In [33]:
test_data, parameters = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")


In [87]:

def SNR_opt(start):
    test_data, parameters = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")

    signal=0
    noise=0
    lower=0
    higher=0
    start=int(start*1000)
    averaged_data = 0 
    x,y =[0,0] 
    averaged_data = average_repeated_data(test_data, 0, start, parameters['idx_pointer'], parameters['reps'])
    x, y = add_fft_trace(averaged_data, parameters['t_ribloc'] + parameters['t_fevol'])

    indicies = np.where((x[3] >= 20) & (x[3] <= 50))[0]  # Get indices where x[3] is between 20 and 50
    lower=np.min(indicies)
    higher= np.max(indicies)

    signal=np.max(y[3][lower:higher])
    noise=calculate_mean_noise_fft(y[3],lower,higher,x)

    if signal/noise!=np.nan:
        print(signal/noise)
        print(start)
        return -signal/noise
    else: 
        return 100

In [88]:
# jarr=np.arange(200,600)
# plt.plot(jarr,SNR)

In [89]:
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.optimize import dual_annealing

In [90]:
bounds=[(20/1000,600/1000)]
# Define a callback function to track progress
# def verbose_callback(x, f, context):
#     """
#     Callback function to print the current guess during optimization.
#     Parameters:
#         x: Current parameter guess.
#         f: Current function value.
#         context: Contextual information (0: initialization, 1: iteration).
#     """
#     if context == 0:
#         print("\nStarting optimization...")
#     else:
#         print(f"Current guess: x = {x:.2f}, Objective function value = {f:.4f}")

# Run the optimizer
print("Running optimization...")
result = dual_annealing(
    SNR_opt,          # Objective function
    bounds=bounds  # Bounds for the variables
    # callback=verbose_callback  # Verbose callback
)

# Check the optimization result and print details
if result.success:
    print("\nOptimization successful!")
    print("Optimal value of x:", result.x)
    print("Objective function value at minimum:", result.fun)
else:
    print("\nOptimization failed!")
    print("Reason for failure:", result.message)

# Additional debug information
print("\nDetailed optimization result:")
print(result)


Running optimization...


C:\Users\HFNV\AppData\Local\Temp\ipykernel_26816\3506998587.py:8: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



2.851743274646417
149
2.8923801016699953
87
15.571131490892649
349
15.571131490892649
349
15.571131490892649
349
15.730163583815107
298
16.646528959853256
374
16.646528959853256
374
16.646528959853256
374
12.223233584482562
240
11.337898794588286
227
11.190866438753453
225
14.984725694709137
341
15.571131490892649
349
17.443461252088767
468
17.443461252088767
468
17.443461252088767
468
17.137123923919763
549
16.420230221296844
597
17.019757962424702
566
17.156537002577164
561
3.0065116305257122
26
14.575418490132895
281
16.519204747594017
382
17.566639705854925
484
17.566639705854925
484
17.566639705854925
484
2.5294714762577404
161
2.491474193663891
160
2.9541505668814465
70
8.913397722620664
189
2.9665207277184833
33
3.1667966392507862
90
2.968583319941225
108
16.600656291441773
377
16.54699633594893
584
17.57045592881068
420
17.57045592881068
420
17.57045592881068
420
17.61327925590999
509
17.019757962424702
566
17.61327925590999
509
17.61327925590999
509
9.548005512423618
196
4.202

KeyboardInterrupt: 

In [39]:
result.x*1000


nan

In [41]:
num=800
data, params = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")

averaged_data = average_repeated_data(data, 0, int(result.x*1000), params['idx_pointer'], params['reps'])
x, y = add_fft_trace(averaged_data, params['t_ribloc'] + params['t_fevol'])
indicies = np.where((x[3] >= 20) & (x[3] <= 50))[0]  # Get indices where x[3] is between 20 and 50
lower=np.min(indicies)
higher= np.max(indicies)
signal=np.max(y[3][lower:higher])
noise=calculate_mean_noise_fft(y[3],lower,higher,x)

fig_fft = go.Figure()

name = ["dark", "bright", "bg", "signal"]
fig_fft.add_trace(go.Scatter(x=x[0], y=y[0], mode='lines', name=name[0]))
fig_fft.add_trace(go.Scatter(x=x[1], y=y[1], mode='lines', name=name[1]))
fig_fft.add_trace(go.Scatter(x=x[2], y=y[2], mode='lines', name=name[2]))
fig_fft.add_trace(go.Scatter(x=x[3], y=y[3], mode='lines', name=name[3]))

fig_fft.update_layout(title='FFT of Signals (Positive Frequencies)',
                      xaxis_title='Frequency [Hz]',
                      yaxis_title='Amplitude')
# Show plots
fig_fft.show()
# noise=calculate_mean_noise_fft(y[3])
signal=np.max(y[3])
print(signal/noise)
np.max(y[3])

C:\Users\HFNV\AppData\Local\Temp\ipykernel_26816\2210882575.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  averaged_data = average_repeated_data(data, 0, int(result.x*1000), params['idx_pointer'], params['reps'])


17.518120916932197


0.06259664275155949

In [58]:
300+5*10+160


510

In [63]:
for i in range(0,1):
    num=10*i
    data, params = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")

    averaged_data = average_repeated_data(data, 0, 510, params['idx_pointer'], params['reps'])
    x, y = add_fft_trace(averaged_data, params['t_ribloc'] + params['t_fevol'])
    indicies = np.where((x[3] >= 20) & (x[3] <= 50))[0]  # Get indices where x[3] is between 20 and 50
    lower=np.min(indicies)
    higher= np.max(indicies)
    signal=np.max(y[3][lower:higher])
    noise=calculate_mean_noise_fft(y[3],lower,higher,x)

    fig_fft = go.Figure()

    name = ["dark", "bright", "bg", "signal"]
    fig_fft.add_trace(go.Scatter(x=x[0], y=y[0], mode='lines', name=name[0]))
    fig_fft.add_trace(go.Scatter(x=x[1], y=y[1], mode='lines', name=name[1]))
    fig_fft.add_trace(go.Scatter(x=x[2], y=y[2], mode='lines', name=name[2]))
    fig_fft.add_trace(go.Scatter(x=x[3], y=y[3], mode='lines', name=name[3]))

    fig_fft.update_layout(title='FFT of Signals (Positive Frequencies)',
                        xaxis_title='Frequency [Hz]',
                        yaxis_title='Amplitude')
    # Show plots
    fig_fft.show()
    # noise=calculate_mean_noise_fft(y[3])
    signal=np.max(y[3])
    print(signal/noise)
    np.max(y[3])

17.628559188734926


In [102]:
def SNR_opt(start):

    test_data, parameters = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")

    start = int(start * 1000)

    # Compute averaged data efficiently
    averaged_data = average_repeated_data(test_data, 0, start, parameters['idx_pointer'], parameters['reps'])
    x, y = add_fft_trace(averaged_data, parameters['t_ribloc'] + parameters['t_fevol'])

    # Use searchsorted for efficient index finding
    lower = np.searchsorted(x[3], 20)
    higher = np.searchsorted(x[3], 50, side="right")

    signal = np.max(y[3][lower:higher])
    noise = calculate_mean_noise_fft(y[3], lower, higher, x)

    snr = signal / noise if noise > 0 else np.nan

    if not np.isnan(snr):
        print(snr, start)
        return -snr
    else:
        return 100

# Read data once to avoid redundant calls

# Define bounds and run optimizer
bounds = [(20 / 1000, 600 / 1000)]
result = dual_annealing(SNR_opt, bounds=bounds,no_local_search=True)

# Print results
if result.success:
    print("Optimization successful!")
    print("Optimal value of x:", result.x)
    print("Objective function value at minimum:", result.fun)
else:
    print("Optimization failed! Reason:", result.message)


C:\Users\HFNV\AppData\Local\Temp\ipykernel_26816\1726259575.py:5: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



8.293291521665868 185
14.307038789786922 275
15.19090394898395 327
15.21907624876521 345
16.24678595378733 357
10.983400779333097 222
17.432398842581776 467
14.475846635705825 279
16.630553158312285 375
15.387164325280105 347
16.049934160003513 307
10.392431502238209 212
2.3431148266575668 82
12.557131309739564 244
4.108896724403125 51
17.051873020912634 538
16.424340889497476 592
6.263432461644842 176
2.9142686017033665 147
17.26069445665153 459
17.63260155773144 510
8.293291521665868 185
15.661107660488687 350
2.5508812051468563 85
16.51810506070621 382
3.1750511525658998 91
16.703687692722458 576
10.286461042464298 210
17.10926266735538 547
17.56195453249479 489
16.02166385208824 354
3.008735473266823 73
17.587644639676817 418
16.627082982876413 366
17.2040816513713 557
17.572705757482062 484
15.97627744583269 303
15.02629836208532 342
17.204900631044136 456
4.506145776712818 56
2.5508812051468563 85
2.8697456019499117 105
2.4492868021071117 159
17.053823483450888 565
11.26604715877

In [95]:
result.x*1000

array([600.])